In [1]:
import json
import re
import numpy as np
import glob

FILENAME_PATTERN = "dataset/perf*.logcat"

def message_to_int(msg):
    match = re.search(r"(\d+)$", msg)
    assert match
    return int(match.group(1))

class Sample:
    def __init__(self, messages):
        assert len(messages) == 6
        if "OpenCV" not in messages[0]:
            raise Exception
        if "Populate Output Array" not in messages[1]:
            raise Exception
        if "Build Parents Array" not in messages[2]:
            raise Exception
        if "Build Tree" not in messages[3]:
            raise Exception
        if "Find Potential Claycodes" not in messages[4]:
            raise Exception
        if "Bit to Text" not in messages[5]:
            raise Exception

        self.openCv = message_to_int(messages[0])
        self.outArray = message_to_int(messages[1])
        self.parentsArray = message_to_int(messages[2])
        self.buildTree = message_to_int(messages[3])
        self.findClaycodes = message_to_int(messages[4])
        self.bitToText = message_to_int(messages[5])

    def get_latency(self):
        return self.bitToText

    def get_topology_extraction_latency(self):
        return self.buildTree

    def get_find_claycodes_latency(self):
        return self.bitToText - self.buildTree
    
# Load all files matching the pattern
log_data = []
for filename in glob.glob(FILENAME_PATTERN):
    with open(filename, "r") as file:
        data = json.load(file)
        latency_messages_slice = [entry["message"] for entry in data["logcatMessages"] if entry["header"]["tag"] == "ClaycodePerformance"]
        log_data.extend(latency_messages_slice)

# Extract performance messages
messages = log_data
msg_header = "Start Decode Process"
samples = []
unexpected_log_lines = 0

for i, msg in enumerate(messages):
    if msg_header in msg:
        next_messages = messages[i + 1: i + 7]
        try:
            sample = Sample(next_messages)
        except Exception:
            # print(f"Unexpected msg after header: {next_messages}")
            unexpected_log_lines += 1
            continue
        samples.append(sample)


latencies = np.array([sample.get_latency() for sample in samples])
print(f"Num latency {len(latencies)}")
print(f"There were {unexpected_log_lines} unexpected log lines")

topology_extranction_latencies = np.array([sample.get_topology_extraction_latency() for sample in samples])
find_claycodes_latency = np.array([sample.get_find_claycodes_latency() for sample in samples])

print(f"Min latency {np.min(latencies)}")
print(f"Avg latency {np.mean(latencies)}")
print(f"Max latency {np.max(latencies)}")
print(f"Std deviation latency {np.std(latencies)}")
print(f"Avg topology extraction latency {np.mean(topology_extranction_latencies)}")
print(f"Avg find claycodes latency {np.mean(find_claycodes_latency)}")


Num latency 10355
There were 47 unexpected log lines
Min latency 42
Avg latency 90.21622404635441
Max latency 196
Std deviation latency 18.065124087358686
Avg topology extraction latency 87.54901014002897
Avg find claycodes latency 2.6672139063254465


In [2]:
import plotly.graph_objects as go

# Create a histogram for latencies
fig = go.Figure()

# Latency histogram
fig.add_trace(go.Histogram(
    x=latencies,
    name='Latencies',
    xbins=dict(size=5),  # Adjust bin size as needed
    opacity=0.75,
    histnorm='percent',
))

# Customize layout
fig.update_layout(
    xaxis_title='Latency (ms)',
    yaxis_title='Frequency (%)',
    barmode='overlay',
    template='plotly_white',
    legend_title='Latency Types',
    margin=dict(t=0, b=0, l=0, r=5),
    font=dict(size=16),
    yaxis=dict(title_standoff=40),
    xaxis=dict(
        range=[35, 203],  # Adjust max as needed to ensure "200" fits
        tickmode='linear',
        tick0=0,
        dtick=20  # You can adjust spacing here
    )
)

# Adjust opacity for better visibility
fig.update_traces(opacity=0.6)

# Show the plot
fig.show()
fig.write_image("latencies.pdf", format="pdf", engine="kaleido")